In [3]:
import os
import spacy
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer



In [7]:
# Cargar el modelo de lenguaje en ingles de SpaCy
nlp = spacy.load('en_core_web_sm')

#es_core_news_sm
# Función de preprocesamiento avanzado
def advanced_preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and not token.is_stop]
    return tokens


# Entrenamiento

In [8]:
# Directorio donde están los archivos
directory = "ArchivosWiki"

# Preparar los datos
sentences = []
contexts = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r", encoding='utf-8') as file:
            content = file.read()
        
        # Dividir el contenido en fragmentos más pequeños
        # Puedes dividir por líneas, párrafos o un número fijo de caracteres
        max_chunk_size = 100000  # Tamaño máximo de cada fragmento
        content_chunks = [content[i:i+max_chunk_size] for i in range(0, len(content), max_chunk_size)]
        
        for chunk in content_chunks:
            # Preprocesar el fragmento
            tokens = advanced_preprocess(chunk)
            sentences.append(tokens)
            
            # Dividir el fragmento en oraciones utilizando SpaCy
            doc = nlp(chunk)
            paragraphs = [sent.text.strip() for sent in doc.sents]
            contexts.extend(paragraphs)

# Entrenar el modelo Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec_wiki.model")

# Cargar el modelo Word2Vec
model = Word2Vec.load("word2vec_wiki.model")

# Crear un DataFrame con los contextos
df = pd.DataFrame({'text': contexts})

# Generar una matriz TF-IDF para ponderar los embeddings
vectorizer = TfidfVectorizer(tokenizer=lambda x: advanced_preprocess(x), lowercase=True)
tfidf_matrix = vectorizer.fit_transform(df['text'])
tfidf_feature_names = vectorizer.get_feature_names_out()

# Crear un diccionario palabra: idf
idf_dict = dict(zip(tfidf_feature_names, vectorizer.idf_))

# Función para obtener el embedding promedio ponderado por TF-IDF de una oración
def get_weighted_sentence_embedding(sentence, model, idf_dict):
    tokens = advanced_preprocess(sentence)
    word_embeddings = []
    weights = []
    for word in tokens:
        if word in model.wv.key_to_index and word in idf_dict:
            word_embeddings.append(model.wv[word])
            weights.append(idf_dict[word])
    if not word_embeddings:
        return np.zeros(model.vector_size)
    word_embeddings = np.array(word_embeddings)
    weights = np.array(weights).reshape(-1,1)
    weighted_average = np.sum(word_embeddings * weights, axis=0) / np.sum(weights)
    return weighted_average

# Generar embeddings para cada contexto
df['embeddings'] = df['text'].apply(lambda x: get_weighted_sentence_embedding(x, model, idf_dict))

# Función para encontrar los contextos más relevantes
def find_most_relevant_contexts(question, df, model, idf_dict, top_n=3):
    question_embedding = get_weighted_sentence_embedding(question, model, idf_dict)
    context_embeddings = np.vstack(df['embeddings'].values)
    similarities = cosine_similarity([question_embedding], context_embeddings)[0]
    df['similarity'] = similarities
    top_indices = df['similarity'].argsort()[-top_n:][::-1]
    most_relevant_contexts = df.iloc[top_indices]['text'].tolist()
    return most_relevant_contexts

# Ejemplo de uso
question = "Who painted the Mona Lisa?"
most_relevant_contexts = find_most_relevant_contexts(question, df, model, idf_dict, top_n=3)

# Imprimir los contextos más relevantes
print("Pregunta:", question)
print("Contextos más relevantes:")
for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")


/home/velasco/Documentos/Proyectos/MachineLearning/Tesis/SistemaQyA/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pregunta: Who painted the Mona Lisa?
Contextos más relevantes:
1. The Mona Lisa wears green in her portrait, as does the bride in the Arnolfini portrait by Jan van Eyck.
2. Biographer Bob Colacello provides some details on Andy's "piss paintings":

Warhol's 1982 portrait of Basquiat, Jean-Michel Basquiat, is a silkscreen over an oxidized copper "piss painting".
3. Saroyan also painted.


# GUARDADO DE INFORMACION

In [9]:
import pickle

# Guardar el DataFrame 'df' con los embeddings
df.to_pickle('df_with_embeddings.pkl')

# Guardar el diccionario 'idf_dict'
with open('idf_dict.pkl', 'wb') as f:
    pickle.dump(idf_dict, f)


In [10]:
# Guardar los embeddings como un archivo numpy
embeddings_array = np.vstack(df['embeddings'].values)
np.save('embeddings.npy', embeddings_array)

# Guardar los textos
#df['text'].to_csv('texts.csv', index=False)


# LECTURA DE LOS ARCHIVOS GENERADOS

In [ ]:
import pickle
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# Cargar el modelo de lenguaje en ingles de SpaCy
nlp = spacy.load('en_core_web_sm')

#es_core_news_sm
# Función de preprocesamiento avanzado
def advanced_preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and not token.is_stop]
    return tokens


# Cargar el modelo de Word2Vec
model = Word2Vec.load("ModelosEmbeddings/word2vec_wiki.model")

# Cargar el DataFrame con los textos y sus embeddings
df = pd.read_pickle('ModelosEmbeddings/df_with_embeddings.pkl')

# Cargar el diccionario 'idf_dict'
with open('ModelosEmbeddings/idf_dict.pkl', 'rb') as f:
    idf_dict = pickle.load(f)

# Cargar los embeddings como un archivo numpy
embeddings_array = np.load('ModelosEmbeddings/embeddings.npy')

# Actualizar los embeddings en el DataFrame (opcional)
df['embeddings'] = list(embeddings_array)

# Función para calcular el embedding de una pregunta
def get_weighted_sentence_embedding(sentence, model, idf_dict):
    tokens = advanced_preprocess(sentence)
    word_embeddings = []
    weights = []
    for word in tokens:
        if word in model.wv.key_to_index and word in idf_dict:
            word_embeddings.append(model.wv[word])
            weights.append(idf_dict[word])
    if not word_embeddings:
        return np.zeros(model.vector_size)
    word_embeddings = np.array(word_embeddings)
    weights = np.array(weights).reshape(-1, 1)
    weighted_average = np.sum(word_embeddings * weights, axis=0) / np.sum(weights)
    return weighted_average

# Función para encontrar los contextos más relevantes
def find_most_relevant_contexts(question, df, model, idf_dict, top_n=3):
    question_embedding = get_weighted_sentence_embedding(question, model, idf_dict)
    context_embeddings = np.vstack(df['embeddings'].values)
    similarities = cosine_similarity([question_embedding], context_embeddings)[0]
    df['similarity'] = similarities
    top_indices = df['similarity'].argsort()[-top_n:][::-1]
    most_relevant_contexts = df.iloc[top_indices]['text'].tolist()
    return most_relevant_contexts




Pregunta: ¿Cuales son los temas principales?
Contextos más relevantes:
1. Tarantino's mother met his father during a trip to Los Angeles.
2. This is known as "culecos" or "los culecos".
3. All of their children were born in Los Angeles.


In [10]:
from googletrans import Translator

# Crear el traductor
translator = Translator()

# Función para traducir la pregunta al inglés
def translate_to_english(question):
    translation = translator.translate(question, src='es', dest='en')
    return translation.text

# Función para traducir texto al español
def translate_to_spanish(text):
    translation = translator.translate(text, src='en', dest='es')
    return translation.text



Pregunta traducida: What is the capital of France?
Pregunta: ¿Cuales son los temas principales?
Contextos más relevantes:
1. "

"Paris is the capital of France.
1. "

"París es la capital de Francia. 



# PREGUNTAS FACTUALES

In [ ]:
# Pregunta 1
spanish_question = "¿Quién pintó la Mona Lisa?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=1)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: Who painted the Mona Lisa?
Pregunta: ¿Cuales son los temas principales?
Contextos más relevantes:
1. The Mona Lisa wears green in her portrait, as does the bride in the Arnolfini portrait by Jan van Eyck.
1. La Mona Lisa usa verde en su retrato, al igual que la novia en el retrato de Arnolfini de Jan Van Eyck. 



In [13]:
# Pregunta 2
spanish_question = "¿Cuál es la capital de Francia?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=1)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is the capital of France?
Contextos más relevantes:
1. "

"Paris is the capital of France.
1. "

"París es la capital de Francia. 



In [16]:
# Pregunta 3
spanish_question = "¿Cuándo comenzó la Segunda Guerra Mundial?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=3)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: When did World War II start?
Contextos más relevantes:
1. It was used in World War II.
1. Fue utilizado en la Segunda Guerra Mundial. 

2. During World War II
2. Durante la Segunda Guerra Mundial 

3. SPs first appeared during World War I, but did not really develop until World War II.
3. SPS apareció por primera vez durante la Primera Guerra Mundial, pero realmente no se desarrolló hasta la Segunda Guerra Mundial. 



In [19]:
# Pregunta 4
spanish_question = "¿Quién fue el primer hombre en caminar sobre la Luna?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=3)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: Who was the first man to walk on the moon?
Contextos más relevantes:
1. In these six spaceflights, twelve people walked on the Moon.
1. En estos seis vuelos espaciales, doce personas caminaron sobre la luna. 

2. Young descended the ladder and, upon setting foot on the lunar surface, became the ninth human to walk on the Moon.
2. Young descendió la escalera y, al poner un pie en la superficie lunar, se convirtió en el noveno humano en caminar sobre la luna. 

3. They walk and walk, with only a white space to be seen all around them.
3. Caminan y caminan, con solo un espacio en blanco para ver a su alrededor. 



In [20]:
# Pregunta 5
spanish_question = "¿Cuál es el río más largo del mundo?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=3)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is the longest river in the world?
Contextos más relevantes:
1. The Amazon River is the widest and second longest river (behind the Nile) in the world.
1. El río Amazon es el río más ancho y largo (detrás del Nilo) del mundo. 

2. The Nile is the longest river in Africa and has historically been considered the longest river in the world, though this has been contested by research suggesting that the Amazon River is slightly longer.
2. El Nilo es el río más largo de África e históricamente ha sido considerado el río más largo del mundo, aunque esto ha sido disputado por una investigación que sugiere que el río Amazonas es un poco más largo. 

3. On the other end of the scale is the tiny Roe River in Montana, which at 201 long is one of the world's shortest rivers.
3. En el otro extremo de la escala está el pequeño río Roe en Montana, que en 201 Long es uno de los ríos más cortos del mundo. 



In [21]:
# Pregunta 6
spanish_question = "¿Qué país tiene la mayor población?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=3)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What country does the largest population have?
Contextos más relevantes:
1. It has a population of 468,095, making it the 90th largest CSA in the nation.
1. Tiene una población de 468,095, lo que lo convierte en el 90º CSA más grande de la nación. 

2. Minnesota has the country's largest Somali population, and the largest Hmong population per capita.
2. Minnesota tiene la población somalí más grande del país y la población hmong más grande per cápita. 

3. With an estimated population of Togo (Year), Togo is the 107th largest country by population.
3. Con una población estimada de Togo (año), Togo es el 107º país más grande por población. 



In [27]:
# Pregunta 7
spanish_question = "¿Quién escribió Cien años de soledad?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=1)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: Who wrote a hundred years of loneliness?
Contextos más relevantes:
1. It was another twelve years before he wrote any more about the Forsytes.
1. Pasaron otros doce años antes de que escribiera más sobre los Forsytes. 



In [32]:
# Pregunta 8
spanish_question = "¿Qué elemento químico tiene el símbolo 'O'?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts("What chemical element has the symbol O?", df, model, idf_dict, top_n=1)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What chemical element does the 'or' symbol have?
Contextos más relevantes:
1. Chemical element symbols (such as Cf for californium and Es for einsteinium), are always capitalized (see below).
1. Los símbolos de elementos químicos (como la FQ para californio y ES para einsteinium), siempre se capitalizan (ver más abajo). 



In [35]:
# Pregunta 9
spanish_question = "¿En qué año se descubrió América?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=3)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What year was America discovered?
Contextos más relevantes:
1. After two years in America, he returned to England dejected and confused.
1. Después de dos años en Estados Unidos, regresó a Inglaterra abatido y confundido. 

2. (He died in America.)
2. (Murió en América). 

3. Within a few years of the publication of his two letters, the European public became aware of the newly discovered continents of the Americas.
3. A los pocos años de la publicación de sus dos cartas, el público europeo se dio cuenta de los continentes recién descubiertos de las Américas. 



In [36]:
# Pregunta 10
spanish_question = "¿Cuál es la montaña más alta del mundo?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=3)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is the highest mountain in the world?
Contextos más relevantes:
1. Further east is the Mahalangur Himal with four of the world's six highest mountains, including the highest: Cho Oyu, Everest, Lhotse, and Makalu.
1. Más al este está el Himal Mahalangur con cuatro de las seis montañas más altas del mundo, incluidas las más altas: Cho Oyu, Everest, Lhotse y Makalu. 

2. The highest mountain in Bhutan is Gangkhar Puensum, which is also a strong candidate for the highest unclimbed mountain in the world.
2. La montaña más alta de Bután es Gangkhar Puensum, que también es un fuerte candidato para la montaña más alta del mundo. 

3. The Himalayas contain the highest mountains in the world, including Mount Everest, which is 8848 high.
3. Los Himalaya contienen las montañas más altas del mundo, incluido el Monte Everest, que tiene 8848 de altura. 



In [38]:
# Pregunta 11
spanish_question = "¿Quién es el autor de la teoría de la relatividad?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: Who is the author of the theory of relativity?
Contextos más relevantes:
1. The phenomenon does not contradict the theory of special relativity.
1. El fenómeno no contradice la teoría de la relatividad especial. 

2. This work of Riemann later became fundamental for Einstein's theory of relativity.
2. Este trabajo de Riemann más tarde se volvió fundamental para la teoría de la relatividad de Einstein. 

3. Examples are Whitehead's theory, Brans–Dicke theory, teleparallelism, f(R) gravity and Einstein–Cartan theory.
3. Ejemplos son la teoría de Whitehead, la teoría de Brans -Dicke, el teleparalelismo, la gravedad F (R) y la teoría de Einstein -Cartan. 

4. The term "theory of relativity" was based on the expression "relative theory" (Relativtheorie) used in 1906 by Planck, who emphasized how the theory uses the principle of relativity.
4. El término "teoría de la relatividad" se basó en la expresión "teoría relativa" (relativ theorie) utilizada en 1906 por Planck, qu

In [39]:
# Pregunta 12
spanish_question = "¿Cuál es el océano más grande del mundo?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is the largest ocean in the world?
Contextos más relevantes:
1. The Atlantic Ocean is the second-largest of the world's five oceans, with an area of about 85133000.
1. El Océano Atlántico es el segundo más grande de los cinco océanos del mundo, con un área de aproximadamente 85133000. 

2. [[Atlantic (disambiguation)]]

The Atlantic Ocean is the second largest of the world's oceans, that separates the old world from the new world.
2. [[Atlantic (desambiguación)]]

El Océano Atlántico es el segundo más grande de los océanos del mundo, que separa el Viejo Mundo del Nuevo Mundo. 

3. This vast pool of salty water is often called the world ocean, and makes Earth with its dynamic hydrosphere a water world or ocean world.
3. Este vasto grupo de agua salada a menudo se llama Océano Mundial, y hace que la Tierra con su hidrosfera dinámica sea un mundo del agua o un mundo oceánico. 

4. The Pacific Ocean is the largest and deepest of Earth's five oceanic divisions.
4. E

In [41]:
# Pregunta 13
spanish_question = "¿Qué país ganó la Copa Mundial de Fútbol 2018?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts("Which country won the 2018 FIFA World Cup?", df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What country did the 2018 World Cup win?
Contextos más relevantes:
1. Ghana has won the Africa Cup of Nations four times, the FIFA U-20 World Cup once, and has participated in three consecutive FIFA World Cups in 2006, 2010, and 2014.
1. Ghana ha ganado la Copa de Naciones de África cuatro veces, la Copa Mundial Sub-20 de la FIFA una vez, y ha participado en tres Copas Mundiales consecutivas de la FIFA en 2006, 2010 y 2014. 

2. The Algeria national football team qualified for the 1982 FIFA World Cup, 1986 FIFA World Cup, 2010 FIFA World Cup and 2014 FIFA World Cup.
2. El equipo de fútbol nacional de Argelia se clasificó para la Copa Mundial de la FIFA de 1982, la Copa Mundial de la FIFA de 1986, la Copa Mundial de la FIFA 2010 y la Copa Mundial de la FIFA 2014. 

3. They won the Africa Cup of Nations in 1980, 1994, and 2013, and have also hosted both the U17 and U20 FIFA World Cup.
3. Ganaron la Copa de Naciones de África en 1980, 1994 y 2013, y también han recibid

In [45]:
# Pregunta 14
spanish_question = "¿Qué planeta es conocido como el Planeta Rojo?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What planet is known as the red planet?
Contextos más relevantes:
1. He is always under some planet".
1. Él siempre está bajo algún planeta ". 

2. The dwarf planet Pluto (formerly considered a planet) is anomalous in several ways, including that it also rotates on its side.
2. El planeta enano Plutón (anteriormente considerado un planeta) es anómalo de varias maneras, incluido que también gira de lado. 

3. Only a handful of giant planets and a handful of minor planets are known around white dwarfs.
3. Solo un puñado de planetas gigantes y un puñado de planetas menores se conocen alrededor de enanos blancos. 

4. Six star systems have been found to have planets: the red dwarf Gliese 832 is one of the closest stars to Earth to have a planetary system.
4. Se ha encontrado que seis sistemas de estrellas tienen planetas: el Red Dwarf Gliese 832 es una de las estrellas más cercanas a la Tierra para tener un sistema planetario. 

5. Many moons of the giant planets have f

In [46]:
# Pregunta 15
spanish_question = "¿Quién fue el primer presidente de los Estados Unidos?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: Who was the first president of the United States?
Contextos más relevantes:
1. It is that you may never be President of the United States."
1. Es que nunca seas presidente de los Estados Unidos ". 

2. [[Vice President of the United States]]

CATEGORIES: Vice presidents of the United States, Vice presidency of the United States, 1789 establishments in the United States, Leaders of the United States Senate, Vice presidents, Articles which contain graphical timelines

The vice president of the United States (VPOTUS) is the second-highest officer in the executive branch of the U.S. federal government, after the president of the United States, and ranks first in the presidential line of succession.
2. [[Vicepresidente de los Estados Unidos]]

Categorías: Vicepresidentes de los Estados Unidos, Vicepresidencia de los Estados Unidos, 1789 establecimientos en los Estados Unidos, líderes del Senado de los Estados Unidos, Vicepresidentes, artículos que contienen líneas de tie

In [56]:
# Pregunta 16
spanish_question = "¿Cuántos colores tiene el arcoíris?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

if most_relevant_contexts:
    for idx, context in enumerate(most_relevant_contexts, 1):
        print(f"{idx}. {context}")
        
else:
    print("No se encontraron contextos relevantes.")

Pregunta traducida: How many colors does the rainbow have?
Contextos más relevantes:
1. They were swaggering in colors...
2. In the RGB color model, named for the colors red, green, and blue, used to create all the colors on a computer screen or television, the range of purples is created by mixing red and blue light of different intensities on a black screen.
3. Other colors in the cyan color range are electric blue, aquamarine, and others described as blue-green.
4. In color printing, purple is sometimes represented by the color magenta, or sometimes by mixing magenta with red or blue.
5. In X11 colors, this color is called both cyan and aqua.
6. In daylight, their color ranges from shades of green, beige, brown, gray, and blue, but in incandescent light, they appear a reddish or purplish/pink color.
7. In the CMY and CMYK color models, red is a secondary color subtractively mixed from magenta and yellow.
8. Color is not one of them."
9. the color.
10. Color is not one of them."


In [ ]:
# Pregunta 17
spanish_question = "¿En qué continente se encuentra Egipto?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What continent is Egypt?
Contextos más relevantes:
1. This makes Egypt a transcontinental country, with the Sinai peninsula in Asia and the remainder of the country in Africa.
1. Esto hace de Egipto un país transcontinental, con la península del Sinaí en Asia y el resto del país en África. 

2. The Atlanteans had conquered the parts of Libya within the Pillars of Hercules, as far as Egypt, and the European continent as far as Tyrrhenia, and had subjected its people to slavery.
2. Los atlantes habían conquistado las partes de Libia dentro de los pilares de Hércules, hasta Egipto y el continente europeo hasta Tirrhenia, y había sometido a su gente a la esclavitud. 

3. [[History of Egypt]]

CATEGORIES: History of Egypt, History of Egypt by period

The  history of Egypt has been long and wealthy, due to the flow of the Nile River with its fertile banks and delta, as well as the accomplishments of Egypt's native inhabitants and outside influence.
3. [[Historia de Egipto

In [61]:
# Pregunta 18
spanish_question = "¿Qué idioma se habla en Brasil?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    

Pregunta traducida: What language is spoken in Brazil?
Contextos más relevantes:
1. An example of such language is Ngwe, which is spoken in Cameroon.
2. They speak the Isan language.
3. 4.11% of Delhites speak other languages.
4. ==Languages==

Portuguese is the official and most widely spoken language of the nation, but in 2017 only 47.4% of Mozambique's population speak Portuguese as either their first or second language, and only 16.6% speak Portuguese as their first language.
5. There is also a translation in Mapudungun, the largest and most-commonly spoken indigenous language in Chile, spoken by the Mapuche people.
6. Part of Ecuador's population can speak Amerindian languages, in some cases as a second language.
7. Part of Ecuador's population can speak Amerindian languages, in some cases as a second language.
8. Hindi is the third most-spoken language in the world including first and second language speakers.
9. The Tuvaluan language is spoken by virtually everyone, while a Micr

In [63]:
# Pregunta 19
spanish_question = "¿Qué moneda se utiliza en Japón?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    

Pregunta traducida: What currency is used in Japan?
Contextos más relevantes:
1. The yen is the official currency of Japan.
2. Its currency, the Hong Kong dollar, is the ninth most traded currency in the world.
3. The Hong Kong dollar, the local currency, is the eighth most traded currency in the world.
4. The Japanese yen is the world's third-largest reserve currency after the US dollar and the euro.
5. So Japan's currency code is JPY: "JP" for Japan and "Y" for yen.
6. The currencies of some countries or regions in the world are freely convertible, such as the US dollar, Australian dollar and Japanese yen.
7. Its currency was the NDH kuna.
8. (See Fiat currency.)
9. [[Currency]]

CATEGORIES: Currency, Foreign exchange market
10. The pound sterling, the currency of the United Kingdom, is globally the fourth-most traded currency and the fourth most held reserve currency.


In [65]:
# Pregunta 20
spanish_question = "¿Qué idioma se habla en Japón?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    

Pregunta traducida: What language is spoken in Japan?
Contextos más relevantes:
1. Japanese is spoken by some older Palauans and is an official language in Angaur.
2. However, in Malaysia as a whole, the majority of ethnic Chinese now speak Mandarin, a non-native language from northern China (originally spoken by the Beijing elite and chosen as the official language of China), as their first language, while English is the first language for the rest.
3. [[Japanese language]]

CATEGORIES: Japanese language, Agglutinative languages, Languages attested from the 8th century, Languages of Japan, Languages of Palau, Subject–object–verb languages

Japanese is the principal language of the Japonic language family spoken by the Japanese people.
4. English is the world's most widely spoken language and the third-most spoken native language.
5. They speak the Isan language.
6. In Australia, Italian is the second most spoken foreign language after Chinese, with 1.4% of the population speaking it a

# PREGUNTAS CONTEXTUALES

In [66]:
# Pregunta 1
spanish_question = "¿Qué significa 'Python' en programación?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What does 'Python' mean in programming?
Contextos más relevantes:
1. ==Structured programming languages==

It is possible to do structured programming in any programming language, though it is preferable to use something like a procedural programming language.
1. == Lenguajes de programación estructurados ==

Es posible realizar una programación estructurada en cualquier lenguaje de programación, aunque es preferible usar algo como un lenguaje de programación de procedimientos. 

2. Examples of pull parsers include Data::Edit::Xml in Perl, StAX in the Java programming language, XMLPullParser in Smalltalk, XMLReader in PHP, ElementTree.iterparse in Python, SmartXML in Red, System.
2. Ejemplos de analgésicos Pull incluyen datos :: Editar :: XML en Perl, Stax en el lenguaje de programación Java, XMLPullParser en SmallTalk, XMLReader en PHP, ElementTree.iterParse en Python, SmartXML en Red, System. 

3. Similarly, the idea of immutable data from functional programming i

In [67]:
# Pregunta 2
spanish_question = "¿Qué significa 'Python' en zoología?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What does 'python' mean in zoology?
Contextos más relevantes:
1. Examples of the use of this prefix in names of Python applications or libraries include Pygame, a binding of SDL to Python (commonly used to create games); PyQt and PyGTK, which bind Qt and GTK to Python respectively; and PyPy, a Python implementation originally written in Python.
1. Los ejemplos del uso de este prefijo en los nombres de las aplicaciones o bibliotecas de Python incluyen Pygame, una vinculación de SDL a Python (comúnmente usado para crear juegos);Pyqt y pygtk, que se unen a Qt y Gtk a Python respectivamente;y Pypy, una implementación de Python originalmente escrita en Python. 

2. I guess I prefer Monty Python in chunk
2. Supongo que prefiero Monty Python en Chunk 

3. The Python programming language, named after Monty Python, prefers to use spam, ham, and eggs as metasyntactic variables, instead of the traditional foo, bar and baz.
3. El lenguaje de programación de Python, que lleva el

In [68]:
# Pregunta 3
spanish_question = "¿Qué es Java en informática?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is Java in computer science?
Contextos más relevantes:
1. Computer science.
1. Ciencias de la Computación. 

2. When a computer has a Java bytecode interpreter, it can run any Java bytecode program, and the same program can be run on any computer that has such an interpreter.
2. Cuando una computadora tiene un intérprete Java Bytecode, puede ejecutar cualquier programa de Bytecode Java, y el mismo programa se puede ejecutar en cualquier computadora que tenga dicho intérprete. 

3. [[Computer science]]

CATEGORIES: Computer science journals, Computer science, Formal sciences

Computer science is the study of computation, information, and automation.
3. [[Ciencias de la Computación]]

Categorías: revistas de informática, informática, ciencias formales

La informática es el estudio de la computación, la información y la automatización. 

4. While at UCLA Crocker taught an extension course on computer programming (for the IBM 7094 mainframe computer).
4. Mientras q

In [69]:
# Pregunta 4
spanish_question = "¿Qué es Java en geografía?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is Java in Geography?
Contextos más relevantes:
1. [[Java (disambiguation)]]

Java is an island of Indonesia.
1. [[Java (desambiguación)]]

Java es una isla de Indonesia. 

2. and Java.
2. y Java. 

3. VOC has established their important bases in some ports in Java, Maluku, and parts of Sulawesi, Sumatra, and Malay Peninsula.
3. VOC ha establecido sus bases importantes en algunos puertos en Java, Maluku y partes de Sulawesi, Sumatra y Península Malaya. 

4. Bali and Java are separated by the Bali Strait.
4. Bali y Java están separados por el Estrecho de Bali. 

5. [[History of Java]]

The History of Java can refer to:

==See also==
5. [[Historia de Java]]

La historia de Java puede referirse a:

== Ver también == 

6. [[Geography of Brunei]]

CATEGORIES: Geography of Brunei

May 2018
6. [[Geografía de Brunei]]

Categorías: Geografía de Brunei

Mayo de 2018 

7. There is no necessary connection between the Java programming language and Java bytecode.
7. No exist

In [70]:
# Pregunta 5
spanish_question = "¿Cuál es la conexión entre el sol y la fotosíntesis?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is the connection between the sun and photosynthesis?
Contextos más relevantes:
1. The evolution of photosynthesis allowed the Sun's energy to be harvested directly by life forms.
1. La evolución de la fotosíntesis permitió que la energía del sol fuera cosechada directamente por formas de vida. 

2. In the Solar System, ice is abundant and occurs naturally from as close to the Sun as Mercury to as far away as the Oort cloud objects.
2. En el sistema solar, el hielo es abundante y ocurre naturalmente desde lo más cerca del sol tan lejos hasta que los objetos de la nube Oort. 

3. Other magnetically related phenomena in the Sun's photosphere include sunspots and solar faculae dispersed between granules.
3. Otros fenómenos relacionados con el sol en la fotosfera del sol incluyen manchas solares y fáulas solares dispersas entre gránulos. 

4. "Preferential absorption of sunlight by ozone over long horizon paths gives the zenith sky its blueness when the sun is near

In [71]:
# Pregunta 6
spanish_question = "¿Cómo afecta el calentamiento global al nivel del mar?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: How does global warming affect at sea level?
Contextos más relevantes:
1. It is anticipated that global warming will result in a rise in sea level.
1. Se anticipa que el calentamiento global dará como resultado un aumento en el nivel del mar. 

2. Since the 1960s, anthropogenic warming of the global ocean combined with contributions of freshwater from retreating land ice causes a global rise in sea level.
2. Desde la década de 1960, el calentamiento antropogénico del océano global combinado con las contribuciones de agua dulce por retirarse al hielo terrestre provoca un aumento global en el nivel del mar. 

3. This has led to increases in mean global temperature, or global warming.
3. Esto ha llevado a aumentos en la temperatura global media o el calentamiento global. 

4. Global warming impact on Kiritimati is thus unpredictable.
4. El impacto del calentamiento global en Kiritimati es, por lo tanto, impredecible. 

5. The United Nations Intergovernmental Panel on C

In [72]:
# Pregunta 7
spanish_question = "¿Qué implica la fusión de los casquetes polares?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What implies the fusion of polar caps?
Contextos más relevantes:
1. As well as novae and dwarf novae, several other classes of these variables are known, including polars and intermediate polars, both of which feature highly magnetic white dwarfs.
1. Además de nova y enano Novae, se conocen varias otras clases de estas variables, incluidos polares y polares intermedios, que presentan enanos blancos altamente magnéticos. 

2. Phases may also be differentiated based on solubility as in polar (hydrophilic) or non-polar (hydrophobic).
2. Las fases también pueden diferenciarse en función de la solubilidad como en la polar (hidrofílica) o no polar (hidrofóbica). 

3. This state of the electrons, called degenerate, meant that a white dwarf could cool to zero temperature and still possess high energy.
3. Este estado de los electrones, llamado degenerado, significaba que un enano blanco podría enfriarse a una temperatura cero y aún poseer alta energía. 

4. ===Protons: polar

In [73]:
# Pregunta 8
spanish_question = "¿Cuál es el impacto del cambio climático en la agricultura?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is the impact of climate change on agriculture?
Contextos más relevantes:
1. This has made agricultural activity vulnerable to climate change and global warming.
1. Esto ha hecho que la actividad agrícola sea vulnerable al cambio climático y el calentamiento global. 

2. [[Causes of climate change]]

CATEGORIES: Climate change, History of climate variability and change, Global environmental issues, Human impact on the environment, Climate change assessment and attribution, Environmental impact of the energy industry, Climatology

The scientific community has been investigating the causes of climate change for decades.
2. [[Causas del cambio climático]]

Categorías: cambio climático, historia de la variabilidad y cambio climático, problemas ambientales globales, impacto humano en el medio ambiente, evaluación y atribución del cambio climático, impacto ambiental de la industria energética, climatología

La comunidad científica ha estado investigando las causas de

In [74]:
# Pregunta 9
spanish_question = "¿Qué beneficios tiene la inteligencia artificial en la medicina?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What benefits does artificial intelligence in medicine have?
Contextos más relevantes:
1. [[Artificial intelligence]]

CATEGORIES: Artificial intelligence, Computational fields of study, Computational neuroscience, Cybernetics, Data science, Formal sciences, Intelligence by type

Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems, as opposed to the natural intelligence of living beings.
1. [[Inteligencia artificial]]

Categorías: inteligencia artificial, campos computacionales de estudio, neurociencia computacional, cibernética, ciencia de datos, ciencias formales, inteligencia por tipo

La inteligencia artificial (IA), en su sentido más amplio, es la inteligencia exhibida por las máquinas, particularmente los sistemas informáticos, en oposición a la inteligencia natural de los seres vivos. 

2. It is most often used for prototyping and artificial intelligence.
2. Se usa con mayor frecuencia para

In [75]:
# Pregunta 10
spanish_question = "¿Qué significa banco en el contexto financiero?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What does bank mean in the financial context?
Contextos más relevantes:
1. For instance the word "bank" is ambiguous since it can refer either to a river bank or to a financial institution, but there are no borderline cases between both interpretations.
1. Por ejemplo, la palabra "banco" es ambigua ya que puede referirse a una orilla del río o a una institución financiera, pero no hay casos límite entre ambas interpretaciones. 

2. In layman's terms, the liability of the central bank is money, and an increase in deposits in member banks carried as a liability by the central bank, means that more money has been put into the economy.
2. En términos de laicos, la responsabilidad del banco central es el dinero, y un aumento en los depósitos en los bancos miembros llevados como un pasivo por parte del banco central significa que se ha puesto más dinero en la economía. 

3. The payment and settlement system and clearing mechanisms were reformed in 2004 through the BCEAO a

In [76]:
# Pregunta 11
spanish_question = "¿Qué significa 'banco' en el contexto de un parque?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What does 'bank' mean in the context of a park?
Contextos más relevantes:
1. Nature reserves around the city include Greetwell Hollow SSSI, Swanholme SSSI, Whisby Nature Park, Boultham Mere and Hartsholme Country Park.
1. Las reservas de la naturaleza en la ciudad incluyen Greetwell Hollow SSSI, Swanholme SSSI, Whisby Nature Park, Boultham Mere y Hartsholme Country Park. 

2. The park was not set aside strictly for ecological purposes.
2. El parque no fue reservado estrictamente para fines ecológicos. 

3. It lies on the banks of the Rhine and is the city's biggest park intra muros.
3. Se encuentra a orillas del Rin y es el parque más grande de la ciudad, Intra Muros. 

4. For instance the word "bank" is ambiguous since it can refer either to a river bank or to a financial institution, but there are no borderline cases between both interpretations.
4. Por ejemplo, la palabra "banco" es ambigua ya que puede referirse a una orilla del río o a una institución financier

In [77]:
# Pregunta 12
spanish_question = "¿Cómo se relaciona la electricidad con el magnetismo?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: How does electricity relate to magnetism?
Contextos más relevantes:
1. He discussed a variety of known forms, which he characterized as common electricity (e.g., static electricity, piezoelectricity, magnetic induction),  voltaic electricity (e.g., electric current from a voltaic pile), and animal electricity (e.g.,  bioelectricity).
1. Discutió una variedad de formas conocidas, que caracterizó como electricidad común (por ejemplo, electricidad estática, piezoelectricidad, inducción magnética), electricidad voltaica (por ejemplo, corriente eléctrica de una pila voltaica) y electricidad animal (por ejemplo, bioelectricidad). 

2. Common phenomena are related to electricity, including lightning, static electricity, electric heating, electric discharges and many others.
2. Los fenómenos comunes están relacionados con la electricidad, que incluyen rayos, electricidad estática, calefacción eléctrica, descargas eléctricas y muchos otros. 

3. ===Electricity and magnetism=

In [78]:
# Pregunta 13
spanish_question = "¿Qué es la computación cuántica y cómo se usa?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is quantum computing and how is it used?
Contextos más relevantes:
1. This led to the advent of quantum computing, which used quantum mechanics to design algorithms.
1. Esto condujo al advenimiento de la computación cuántica, que utilizaba mecánica cuántica para diseñar algoritmos. 

2. ====Neuromorphic quantum computing====

Neuromorphic quantum computing (abbreviated as ‘n.quantum computing’) is an unconventional computing type of computing that uses neuromorphic computing to perform quantum operations.
2. ==== Computación cuántica neuromórfica ====

La computación cuántica neuromórfica (abreviada como "N.Quantum Computing") es un tipo de computación no convencional que utiliza la computación neuromórfica para realizar operaciones cuánticas. 

3. Neuromorphic quantum computing and quantum computing share similar physical properties during computation.
3. La computación cuántica neuromórfica y la computación cuántica comparten propiedades físicas similares dur

In [79]:
# Pregunta 14
spanish_question = "¿Cómo se define la criptomoneda?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: How is cryptocurrency defined?
Contextos más relevantes:
1. These are defined by PICMG 2.0R3 and PICMG Exp0
1. Estos se definen por picmg 2.0r3 y picmg exp0 

2. IMAP is defined by 9051.
2. IMAP se define por 9051. 

3. DD *

DEFINE GDG (NAME('DB2.FULLCOPY.DSNDB04.TSTEST') LIMIT(5)
3. DD *

Define GDG (nombre ('db2.fullCopy.DSNDB04.tstest') Límite (5) 

4. For example, define:
4. Por ejemplo, defina: 

5. These include the TCXO, MCXO, and OCXO which are defined below.
5. Estos incluyen TCXO, MCXO y OCXO que se definen a continuación. 

6. Y is standard and defined above, and can also be defined as BU(CBU), so that g(Yg).
6. Y es estándar y definido anteriormente, y también se puede definir como BU (CBU), de modo que G (yg). 

7. This defines a sieve.
7. Esto define un tamiz. 

8. The base units are defined in terms of the defining constants.
8. Las unidades base se definen en términos de las constantes definitorias. 

9. The term is not elsewhere defined in the rule

In [80]:
# Pregunta 15
spanish_question = "¿Qué papel desempeña el agua en el ciclo de vida?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What role does water play in the life cycle?
Contextos más relevantes:
1. Whales play an important role in the carbon cycle in life and death.
1. Las ballenas juegan un papel importante en el ciclo del carbono en la vida y la muerte. 

2. The last stage of the life cycle is the end of life.
2. La última etapa del ciclo de vida es el final de la vida. 

3. Food also plays a role in daily life.
3. La comida también juega un papel en la vida diaria. 

4. Sufism continues to play an important role in the daily lives of Sindhis.
4. El sufismo continúa desempeñando un papel importante en la vida cotidiana de Sindhis. 

5. It is thought that they played a key role in enabling life on Earth and its emergence.
5. Se cree que jugaron un papel clave para permitir la vida en la tierra y su surgimiento. 

6. Water changes its state of being several times throughout this cycle.
6. El agua cambia su estado de ser varias veces a lo largo de este ciclo. 

7. Most cereals need regula

In [81]:
# Pregunta 16
spanish_question = "¿Qué significa la Revolución Industrial en términos históricos?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What does the industrial revolution mean in historical terms?
Contextos más relevantes:
1. One of the classic examples of the usage of the word revolution in such context is the Industrial Revolution, Scientific Revolution or the Commercial Revolution.
1. Uno de los ejemplos clásicos del uso de la palabra revolución en dicho contexto es la revolución industrial, la revolución científica o la revolución comercial. 

2. [[Industrial Revolution]]

CATEGORIES: Industrial Revolution, 18th century in technology, 19th century in technology, Age of Revolution, Early Modern Britain, History of technology, Industrial history, Late modern Europe, Modern history of the United Kingdom, Revolutions by type, Stages of history

The Industrial Revolution, also known as the First Industrial Revolution,  was a period of global transition of the human economy towards more widespread, efficient and stable manufacturing processes that succeeded the Agricultural Revolution, starting from 

In [82]:
# Pregunta 17
spanish_question = "¿Cuál es la relación entre el capitalismo y el libre mercado?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is the relationship between capitalism and free market?
Contextos más relevantes:
1. Murray Rothbard stated that the difference between free-market capitalism and state capitalism is the difference between "peaceful, voluntary exchange" and a "collusive partnership" between business and government that "uses coercion to subvert the free market".
1. Murray Rothbard declaró que la diferencia entre el capitalismo de libre mercado y el capitalismo estatal es la diferencia entre "intercambio pacífico y voluntario" y una "asociación colusiva" entre los negocios y el gobierno que "utiliza la coerción para subvertir el libre mercado". 

2. They see capitalism and the "free market" as the basis for a free and prosperous society.
2. Ven el capitalismo y el "libre mercado" como la base de una sociedad libre y próspera. 

3. These two contradictory concepts are what I would call 'free-market capitalism' on the one hand, and 'state capitalism' on the other".
3. Estos dos co

In [83]:
# Pregunta 18
spanish_question = "¿Qué significa 'blockchain' en el contexto tecnológico?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What does 'blockchain' mean in the technological context?
Contextos más relevantes:
1. Technological determinism is the idea that technologies cause unavoidable social changes.95 It usually encompasses a related argument, technological autonomy, which asserts that technological progress follows a natural progression and cannot be prevented.
1. El determinismo tecnológico es la idea de que las tecnologías causan cambios sociales inevitables.95 Por lo general, abarca un argumento relacionado, autonomía tecnológica, lo que afirma que el progreso tecnológico sigue una progresión natural y no se puede prevenir. 

2. However, Haraway has distanced herself from posthumanistic discourse due to other theorists' use of the term to promote utopian views of technological innovation to extend the human biological capacity (even though these notions would more correctly fall into the realm of transhumanism).
2. Sin embargo, Haraway se ha distanciado del discurso posthumanista deb

In [84]:
# Pregunta 19
spanish_question = "¿Cuál es la importancia de la fotosíntesis en la naturaleza?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What is the importance of photosynthesis in nature?
Contextos más relevantes:
1. The importance of fungi in Tasmania's ecology is often overlooked, but nonetheless they play a vital role in the natural vegetation cycle.
1. La importancia de los hongos en la ecología de Tasmania a menudo se pasa por alto, pero no obstante juegan un papel vital en el ciclo de la vegetación natural. 

2. The biological importance of the accumulation is unknown.
2. Se desconoce la importancia biológica de la acumulación. 

3. Biological oceanography mainly focuses on the ecosystem of the ocean with an emphasis on plankton: their diversity (morphology, nutritional sources, motility, and metabolism); their productivity and how that plays a role in the global carbon cycle; and their distribution (predation and life cycle).
3. La oceanografía biológica se centra principalmente en el ecosistema del océano con énfasis en el plancton: su diversidad (morfología, fuentes nutricionales, motilidad

In [85]:
# Pregunta 20
spanish_question = "¿Qué papel juega la educación en el desarrollo económico?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Ahora puedes pasar la pregunta traducida al modelo
most_relevant_contexts = find_most_relevant_contexts(english_question, df, model, idf_dict, top_n=10)

# Imprimir los contextos más relevantes
print("Contextos más relevantes:")

for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")
    translated_text = translate_to_spanish(context)
    print(f"{idx}. {translated_text} \n")

Pregunta traducida: What role does education in economic development play?
Contextos más relevantes:
1. School infrastructure also plays a crucial role in educational success.
1. La infraestructura escolar también juega un papel crucial en el éxito educativo. 

2. Education played a major role in the socio-economic transformation of the archipelago.
2. La educación jugó un papel importante en la transformación socioeconómica del archipiélago. 

3. As a social institution, education plays an integral role in the process of socialization.
3. Como institución social, la educación juega un papel integral en el proceso de socialización. 

4. Geography and education levels play major roles therein.
4. Los niveles de geografía y educación juegan papeles importantes allí. 

5. Organized institutions play a significant role in education.
5. Las instituciones organizadas juegan un papel importante en la educación. 

6. Kolkata has played a pioneering role in the development of the modern educati

# PREGUNTAS ANALOGICAS

In [94]:
def find_analogy(word_a, word_b, word_c, model, top_n=1):
    """
    Encuentra la palabra que completa la analogía: word_a es a word_b como word_c es a ?
    """
    # Verificar si las palabras están en el vocabulario
    for word in [word_a, word_b, word_c]:
        if word not in model.wv.key_to_index:
            return f"'{word}' no está en el vocabulario del modelo."
    
    # Calcular el vector resultante para la analogía
    analogy_vector = model.wv[word_a] - model.wv[word_b] + model.wv[word_c]
    
    # Encontrar las palabras más similares al vector resultante
    similar_words = model.wv.similar_by_vector(analogy_vector, topn=top_n)
    
    return [word for word, similarity in similar_words]


In [97]:
# Pregunta 1
spanish_question = "Rey es a reina como hombre es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "king", "queen", "man" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: King is queen as a man is what?
['king', 'is', 'queen', 'as', 'a', 'man', 'is', 'what?']
king es a queen como man es a: ['man', 'soldier', 'warrior']


In [99]:
# Pregunta 2
spanish_question = "Madrid es a España como Tokio es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "madrid", "spain", "tokio" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: Madrid is Spain as Tokyo is what?
['madrid', 'is', 'spain', 'as', 'tokyo', 'is', 'what?']
madrid es a spain como tokio es a: ['chelsea', 'fulham', 'luton']


In [101]:
# Pregunta 3
spanish_question = "Día es a sol como noche es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "day", "sunny", "night" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: Day is sunny as night is what?
['day', 'is', 'sunny', 'as', 'night', 'is', 'what?']
day es a sunny como night es a: ['day', 'night', 'week']


In [103]:
# Pregunta 4
spanish_question = "La cuchara es a sopa como tenedor es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "spoon", "soup", "fork" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: The spoon is a soup as a fork is what?
['the', 'spoon', 'is', 'a', 'soup', 'as', 'a', 'fork', 'is', 'what?']
spoon es a soup como fork es a: ['fork', 'block', 'fishermans']


In [105]:
# Pregunta 5
spanish_question = "Manzana es a fruta como perro es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)


# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "apple", "fruit", "dog" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: Apple is fruit as a dog is what?
['apple', 'is', 'fruit', 'as', 'a', 'dog', 'is', 'what?']
apple es a fruit como dog es a: ['apple', 'kermit', 'trs-80']


In [107]:
# Pregunta 6
spanish_question = "Ojo es a ver cómo oreja es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "eye", "see", "ear" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: Eye is to see how ear is what?
['eye', 'is', 'to', 'see', 'how', 'ear', 'is', 'what?']
eye es a see como ear es a: ['ear', 'eye', 'eyelid']


In [109]:
# Pregunta 7
spanish_question = "El carro es a combustible como humano es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "car", "fuel", "human" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: The car is fuel as a human is what?
['the', 'car', 'is', 'fuel', 'as', 'a', 'human', 'is', 'what?']
car es a fuel como human es a: ['human', 'person', 'individual']


In [111]:
# Pregunta 8
spanish_question = "Ave es a volar como pez es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "bird", "flying", "fish" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: Bird is flying as a fish is to what?
['bird', 'is', 'flying', 'as', 'a', 'fish', 'is', 'to', 'what?']
bird es a flying como fish es a: ['fish', 'bird', 'salmon']


In [113]:
# Pregunta 9
spanish_question = "Computadora es a procesador como humano es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "computer", "processor", "human" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: Computer is a processor as a human is what?
['computer', 'is', 'a', 'processor', 'as', 'a', 'human', 'is', 'what?']
computer es a processor como human es a: ['human', 'animal', 'sentient']


In [115]:
# Pregunta 10
spanish_question = "Libro es a leer como película es a ¿qué?"
english_question = translate_to_english(spanish_question)
print("Pregunta traducida:", english_question)

# Extraer las palabras de la pregunta traducida
words = english_question.lower().split()
print(words)
word_a, word_b, word_c = "book", "read", "movie" # Palabras Extraidas

# Resolver la analogía
result = find_analogy(word_a, word_b, word_c, model, top_n=3)


# Mostrar los resultados
print(f"{word_a} es a {word_b} como {word_c} es a: {result}")

Pregunta traducida: Book is to read as a movie is what?
['book', 'is', 'to', 'read', 'as', 'a', 'movie', 'is', 'what?']
book es a read como movie es a: ['movie', 'film', 'sequel']
